<a href="https://colab.research.google.com/github/marcosoares-92/IndustrialDataScience_DataExtraction/blob/main/1_Data_Extraction_IP21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Extraction from Aspen InfoPlus21 (IP21)**

## _Data Extraction Workflow Notebook 1_

## Content:
1. Retrieving a list of dataframes returned from each API call;
2. Fetching or updating a SQLite database;
3. Mounting the storage system (S3 or Google Drive);
4. Loading previously obtained Pandas dataframe;
5. Concatenating (SQL UNION) multiple dataframes;
6. Removing duplicate rows from the dataframe;
7. Exporting the dataframe as CSV File (to notebook's workspace);
8. Importing or exporting models, lists, or dictionaries;
9. Downloading a file from Google Colab to the local machine; or uploading a file from the machine to Colab's instant memory;
10. Exporting a list of files from notebook's workspace to AWS Simple Storage Service (S3).

### Attention

IP21 API runs only in Firefox and Chrome. If Edge or other browser is being used, run the following command in a cell:

- `!jupyter notebook list`

- or run `jupyter notebook list` in the Anaconda command line.

This will show an URL like:
    
`http://localhost:8888/?token=TOKEN_VALUE :: C:\Users\Dir`

- Then, open your Firefox, Chrome, or other browser, access the page http://localhost:8888, and copy and paste the token represented by `TOKEN_VALUE` in the correspondent input box.

Marco Cesar Prado Soares, Data Scientist Specialist - Bayer Crop Science LATAM
- marcosoares.feq@gmail.com
- marco.soares@bayer.com

## **Load Python Libraries in Global Context**

In [ ]:
import load
from idsw import *

## **Call the functions**

### **Retrieving a list of dataframes returned from each API call**

In [ ]:
IP21_SERVER = 'ip21_server_name'
# IP21_SERVER is a string informing the server name for the IP21 REST API.
# If you check ASPEN ONE or ASPEN IP21 REST API URL, it will have a format like:
# http://ip21_server_name/ProcessData/AtProcessDataREST.dll/
# or like:
# http://ip21_server_name.company_website/processexplorer/aspenONE.html
# In this case, declare:
# IP21_SERVER = 'ip21_server_name' or as 'ip21_server_name/'

LIST_OF_TAGS_TO_EXTRACT = [{'tag': None, 'actual_name': None}]
# LIST_OF_TAGS_TO_EXTRACT = [{'tag': None, 'actual_name': None}] is a list of dictionaries.
# The dictionaries should have always the same keys: 'tag', containing the tag name as registered
# in the system, and 'actual_name', with a desired name for the variable. You can add as much
# tags as you want, but adding several tags may lead to a blockage by the server. The key 'actual_name'
# may be empty, but dictionaries where the 'tag' value is None will be ignored.
# Examples: LIST_OF_TAGS_TO_EXTRACT = [{'tag': 'TEMP', 'actual_name': 'temperature'}]
# LIST_OF_TAGS_TO_EXTRACT = [{'tag': 'TEMP2.1.2', 'actual_name': 'temperature'},
# {'tag': 'PUMP.1.2', 'actual_name': 'pump_pressure'}, {'tag': 'PHTANK', 'actual_name': 'ph'}]
# LIST_OF_TAGS_TO_EXTRACT = [{'tag': 'TEMP', 'actual_name': None}]

USERNAME = None
PASSWORD = None
# USERNAME = None, PASSWORD = None: declare your username and password as strings (in quotes)
# or keep username = None, password = None to generate input boxes. The key typed on the boxes
# will be masked, so other users cannot see it.

DATA_SOURCE = 'localhost'
# DATA_SOURCE = 'localhost': string informing the particular data source to fetch on IP21.
# Keep DATA_SOURCE = 'localhost' to query all available data sources.

START_TIME = {'year': 2015, 'month': 1, 'day':1, 'hour': 0, 'minute': 0, 'second': 0}
STOP_TIME = {'year': 2022, 'month': 4, 'day': 1, 'hour': 0, 'minute': 0, 'second': 0}
# START_TIME: dictionary containing start timestamp information.
# Example: START_TIME = {'year': 2015, 'month': 1, 'day':1, 'hour': 0, 'minute': 0, 'second': 0}
# STOP_TIME: dictionary containing stop timestamp information.
# Example: STOP_TIME = {'year': 2022, 'month': 4, 'day': 1, 'hour': 0, 'minute': 0, 'second': 0}

# Alternatively: START_TIME = 'today', 'now', START_TIME = 'yesterday', START_TIME = -10 for 10
# days before, START_TIME = -X for - X days before. Units for offsets will be always in days, unless
# you modify the parameters START_TIMEDELTA_UNIT and STOP_TIMEDELTA_UNIT.
# For the timedelta unit, set 'day' or 'd' for subtracting values in days,'hour' or 'h',
# 'minute' or 'm' for minutes, 'second' or 's' for seconds, 'milisecond' or 'ms' for miliseconds.
# Put the "-" signal, or the time will be interpreted as a future day from today.
# Analogously for stop_time.
# Both dictionaries must contain only float values (for 'year', 'day' and 'month' are integers,
# naturally).

## WARNING: The keys must be always be the same, only change the numeric values.
## The keys must be: 'year', 'month', 'day', 'hour', 'minute', and 'second'

START_TIMEDELTA_UNIT = 'day'
STOP_TIMEDELTA_UNIT = 'day'
# START_TIMEDELTA_UNIT = 'day'
# If START_TIME was declared as a numeric value (integer or float), specify the timescale units
# in this parameter. The possible values are: 'day' or 'd'; 'hour' or 'h'; 'minute' or 'm';
# 'second' or 's', 'milisecond' or 'ms'.
# STOP_TIMEDELTA_UNIT = 'day' - analogous to START_TIMEDELTA_UNIT. Set this parameter when
# declaring STOP_TIME as a numeric value.

IP21TIME_ARRAY = []
# IP21TIME_ARRAY = [] - keep this parameter as an empty list or set IP21TIME_ARRAY = None.
# If you want to use the method to independently convert an array, you could pass this array
# to the constructor to convert it.

PREVIOUS_DATAFRAME_FOR_CONCATENATION = None
# PREVIOUS_DATAFRAME_FOR_CONCATENATION = None: keep it None or, if you want to append the fetched data
# to a pre-existing database, declare the object containing the pandas dataframe where it will
# be appended. Example: PREVIOUS_DATAFRAME_FOR_CONCATENATION = dataset.


# list of dataframes returned from each API call returned as returned_dfs_list.
# Simply modify this object on the left of the equality
returned_dfs_list = get_data_from_ip21 (ip21_server = IP21_SERVER, list_of_tags_to_extract = LIST_OF_TAGS_TO_EXTRACT, username = USERNAME, password = PASSWORD, data_source = DATA_SOURCE, start_time = START_TIME, stop_time = STOP_TIME, start_timedelta_unit = START_TIMEDELTA_UNIT, stop_timedelta_unit = STOP_TIMEDELTA_UNIT, ip21time_array = IP21TIME_ARRAY, previous_df_for_concatenation = PREVIOUS_DATAFRAME_FOR_CONCATENATION)

### **Fetching or updating a SQLite database**

In [ ]:
FILE_PATH = '/my_db.db'
# FILE_PATH: full path of the SQLite file. It may start with './' or '/',
# but with no more than 2 slashes.
# It is a string: input in quotes. Example: FILE_PATH = '/my_db.db'

TABLE_NAME = 'main_table'
# TABLE_NAME: string with the name of the table that will be fetched or updated.
# Example: TABLE_NAME = 'main_table'

ACTION = 'fetch_table'
# ACTION = 'fetch_table' to access a table named TABLE_NAME from the database.
# ACTION = 'update_table' to update a table named TABLE_NAME from the database.

PRE_CREATED_ENGINE = None
# PRE_CREATED_ENGINE = None - if None, a new engine will be created. If an engine was already created,
# pass it as argument: PRE_CREATED_ENGINE = engine

DATASET = None
# DATASET = None - if a table is going to be updated, input here the new Pandas dataframe
# (object) correspondent to the table.
# Example: DATASET = dataset.


# Dataframe correspondent to the fetched (or updated) table returned as sqlite_df.
# Engine used for manipulating the database returned as engine.
# Simply modify these objects in the left of the equality:
sqlite_df, engine = manipulate_sqlite_db (file_path = FILE_PATH, table_name = TABLE_NAME, action = ACTION, pre_created_engine = PRE_CREATED_ENGINE, df = DATASET)

## **Exporting the dataframe as CSV file (to notebook's workspace)**

In [ ]:
## WARNING: all files exported from this function are .csv (comma separated values)

DATAFRAME_OBJ_TO_BE_EXPORTED = dataset
# Alternatively: object containing the dataset to be exported.
# DATAFRAME_OBJ_TO_BE_EXPORTED: dataframe object that is going to be exported from the
# function. Since it is an object (not a string), it should not be declared in quotes.
# example: DATAFRAME_OBJ_TO_BE_EXPORTED = dataset will export the dataset object.
# ATTENTION: The dataframe object must be a Pandas dataframe.

FILE_DIRECTORY_PATH = ""
# FILE_DIRECTORY_PATH - (string, in quotes): input the path of the directory
# (e.g. folder path) where the file is stored. e.g. FILE_DIRECTORY_PATH = ""
# or FILE_DIRECTORY_PATH = "folder"
# If you want to export the file to AWS S3, this parameter will have no effect.
# In this case, you can set FILE_DIRECTORY_PATH = None

NEW_FILE_NAME_WITHOUT_EXTENSION = "dataset"
# NEW_FILE_NAME_WITHOUT_EXTENSION - (string, in quotes): input the name of the
# file without the extension. e.g. set NEW_FILE_NAME_WITHOUT_EXTENSION = "my_file"
# to export the CSV file 'my_file.csv' to notebook's workspace.

export_pd_dataframe_as_csv (dataframe_obj_to_be_exported = DATAFRAME_OBJ_TO_BE_EXPORTED, new_file_name_without_extension = NEW_FILE_NAME_WITHOUT_EXTENSION, file_directory_path = FILE_DIRECTORY_PATH)

## **Exporting dataframes as Excel file tables**

In [ ]:
## WARNING: all files exported from this function are .xlsx

FILE_NAME_WITHOUT_EXTENSION = "datasets"
# (string, in quotes): input the name of the
# file without the extension. e.g. new_file_name_without_extension = "my_file"
# will export a file 'my_file.xlsx' to notebook's workspace.

EXPORTED_TABLES = [{'dataframe_obj_to_be_exported': None,
                    'excel_sheet_name': None},]

# exported_tables is a list of dictionaries. User may declare several dictionaries,
# as long as the keys are always the same, and if the values stored in keys are not None.

# key 'dataframe_obj_to_be_exported': dataframe object that is going to be exported from the
# function. Since it is an object (not a string), it should not be declared in quotes.
# example: dataframe_obj_to_be_exported = dataset will export the dataset object.
# ATTENTION: The dataframe object must be a Pandas dataframe.

# key 'excel_sheet_name': string containing the name of the sheet to be written on the
# exported Excel file. Example: excel_sheet_name = 'tab_1' will save the dataframe in the
# sheet 'tab_1' from the file named as file_name_without_extension.

# examples: exported_tables = [{'dataframe_obj_to_be_exported': dataset1,
# 'excel_sheet_name': 'sheet1'},]
# will export only dataset1 as 'sheet1';
# exported_tables = [{'dataframe_obj_to_be_exported': dataset1, 'excel_sheet_name': 'sheet1'},
# {'dataframe_obj_to_be_exported': dataset2, 'excel_sheet_name': 'sheet2']
# will export dataset1 as 'sheet1' and dataset2 as 'sheet2'.

# Notice that if the file does not contain the exported sheets, they will be created. If it has,
# the sheets will be replaced.

FILE_DIRECTORY_PATH = ""
# FILE_DIRECTORY_PATH - (string, in quotes): input the path of the directory
# (e.g. folder path) where the file is stored. e.g. FILE_DIRECTORY_PATH = ""
# or FILE_DIRECTORY_PATH = "folder"
# If you want to export the file to AWS S3, this parameter will have no effect.
# In this case, you can set FILE_DIRECTORY_PATH = None


export_pd_dataframe_as_excel (file_name_without_extension = FILE_NAME_WITHOUT_EXTENSION, exported_tables = EXPORTED_TABLES, file_directory_path = FILE_DIRECTORY_PATH)

****

# **Scope in Azure Active Directory (AD) - What is a scope in azure security - Background**
- AD: Active Directory allows network administrators to create and manage domains, users, and objects within a network. For example, an admin can create a group of users and give them specific access privileges to certain directories on the server.
- The AD is the database that contains the security information and permissions of the users.
- By limiting the scope, you limit what resources are at risk if the security principal is ever compromised. In Azure, you can specify a scope at four levels: management group, subscription, resource group, and resource. Scopes are structured in a parent-child relationship.
#### Therefore, the Azure scope represents the security privileges of an user.